## Citibike Dataset



In [2]:
%matplotlib inline
import pandas as pd
import MySQLdb as mdb
import matplotlib 
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
matplotlib.rcParams['figure.figsize'] = (20,10)

In [4]:
from sqlalchemy import create_engine

conn_string_read = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'localhost', 
    user = 'root',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4', db='citibike')
engine_read = create_engine(conn_string_read)



In [8]:
query = '''
SELECT station_id AS id, 
       AVG(available_bikes) AS available_bikes,
       AVG(available_docks) AS available_docks,
       AVG(available_bikes/(available_bikes+available_docks)) AS percent_full, 
       DATE_FORMAT(last_communication_time, '%%Y-%%m-%%d %%H:00:00') AS communication_time 
FROM citibike.Docks_Status
WHERE  last_communication_time > '2017-02-13' AND last_communication_time < '2017-03-13' 
    AND status_key=1 
    AND available_bikes+available_docks>0
    
GROUP BY station_id, communication_time

'''
df = pd.read_sql(query, con=engine_read)

In [9]:
len(df)

433343

In [16]:
df.head(10)

,id,available_bikes,available_docks,percent_full,communication_time
0,72,13.0000,26.0000,0.333300,2017-02-13 00:00:00
1,72,13.0000,26.0000,0.333300,2017-02-13 01:00:00
2,72,13.0000,26.0000,0.333300,2017-02-13 02:00:00
3,72,13.0000,26.0000,0.333300,2017-02-13 03:00:00
4,72,13.0000,26.0000,0.333300,2017-02-13 04:00:00
5,72,13.0000,26.0000,0.333300,2017-02-13 05:00:00
6,72,12.4667,26.5333,0.319647,2017-02-13 06:00:00
7,72,11.8824,27.1176,0.304688,2017-02-13 07:00:00
8,72,11.4286,27.5714,0.293043,2017-02-13 08:00:00
9,72,15.7727,23.2273,0.404436,2017-02-13 09:00:00


In [18]:
df['communication_time'] = pd.to_datetime(df['communication_time'], format='%Y-%m-%d %H:%M:%S')

In [19]:
df['total_docks'] =  df.available_bikes + df.available_docks

In [21]:
df.total_docks = df.total_docks.astype(int)

In [23]:
df = df[ ['id', 'communication_time', 'available_bikes', 'available_docks', 'total_docks', 'percent_full' ] ]

In [25]:
df.sort_values(by = ['id', 'communication_time'], inplace=True)

In [27]:
df.dtypes

id                             int64
communication_time    datetime64[ns]
available_bikes              float64
available_docks              float64
total_docks                    int64
percent_full                 float64
dtype: object

### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [10]:
conn_string_write = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'krjniJm6OuIyhEoo',
    encoding = 'utf8mb4')
engine_write = create_engine(conn_string_write)


Once we have connected successfully, we need to create our database:

In [12]:
# Query to create a database
db_name = 'citibike'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8mb4'".format(db=db_name)

# Create a database
engine_write.execute(create_db_query)
# And lets switch to the database
engine_write.execute("USE {db}".format(db=db_name))

In [29]:
table_name = 'stations'

df.to_sql(
    name = table_name, 
    schema = db_name, 
    con = engine_write,
    if_exists = 'replace',
    index = False, 
    chunksize = 1000)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
# add_key_query = 'ALTER TABLE nypd ADD PRIMARY KEY(CMPLNT_NUM)'
# engine.execute(add_key_query)